In [1]:
import sqlite3
from sqlite3 import Error
import os
import bcrypt

# Criar tabelas

In [2]:
#CRIA A CONEXÃO COM O BANCO DE DADOS
def create_connection(db_file):
    """ Cria conexão com o banco de dados SQLite
        especificado por arquivo_db
        :param arquivo_db: diretório do banco de dados
        :return : Objeto de conexão ou None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        return e

    return conn

In [3]:
# CRIA UMA FUNÇÃO GERADORA DE TABELAS
def create_table(conn, create_table_sql):

    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        return e

In [4]:
def main():
    database = "fluxo.db"

    sql_create_movimentacoes_table = """CREATE TABLE IF NOT EXISTS movimentacoes(
                                    id_movimentacao INTEGER primary key autoincrement,
                                    data DATE not null,
                                    tipo TEXT(50) not null,
                                    id_categoria INTEGER not null,
                                    id_conta INTEGER not null,
                                    comentario TEXT(300),
                                    valor REAL not null
                                ); """
    

    sql_create_conta_table = """CREATE TABLE IF NOT EXISTS conta(
                                id_conta INTEGER primary key autoincrement,
                                nome TEXT(100) not null,
                                saldo REAL
                                ); """
    
    sql_create_categoria = """CREATE TABLE IF NOT EXISTS categoria(
                                id_categoria INTEGER primary key autoincrement,
                                tipo TEXT(50) not null,
                                nome TEXT(50) not null
                                )"""
    sql_create_password = """CREATE TABLE IF NOT EXISTS password(
                                password 
                                )"""
    
    # Cria a conexão com o banco de dados
    conn = create_connection(database)

    # Cria as tabelas
    if conn is not None:
        
        # cria a tabela movimentações
        create_table(conn, sql_create_movimentacoes_table)

        # cria a tabela conta
        create_table(conn, sql_create_conta_table)
        
        # cria a tabela categoria
        create_table(conn, sql_create_categoria)
        
        # cria a tabela password
        create_table(conn, sql_create_password)
        
        
    else:
        print("Error! cannot create the database connection.")



In [6]:
main()

# Insert

In [10]:
def adicionar_conta(nome : str):
    database = "fluxo.db"
    conn = create_connection(database)
    sql = ''' INSERT INTO conta(nome)
              VALUES(?)'''
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, (nome,))
            conn.commit()
            return cur.lastrowid
        
        except Error as e:
            return e

In [11]:
adicionar_conta("Neon")

1

In [9]:
def adicionar_senha(senha : str):
    database = "fluxo.db"
    conn = create_connection(database)
    sql = ''' INSERT INTO password(password)
              VALUES(?)'''
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql, (senha,))
            conn.commit()
            return cur.lastrowid
        
        except Error as e:
            return e

# SELECT 

In [11]:
# Função para que as consultas SQL retornem uma lista de dicionários ao invés de uma lista de tuplas
def dict_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    return {key: value for key, value in zip(fields, row)}


In [18]:
def consultar_senha():
    database = "fluxo.db"
    conn = create_connection(database)
    conn.row_factory = dict_factory
    
    sql = "SELECT * FROM password"
    
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(sql)
            consulta = cur.fetchone()
            
            return consulta
            
        except Error as e:
            return e
consultar_senha()

{'password': 7593258303979883720}

# Testes 

In [12]:
os.path.exists("fluxo.db")

True

In [24]:
word = "johnny*03"
print(hash(word))

senha = hash(word)
adicionar_senha(senha)

result = consultar_senha()

if result['password'] == senha:
    print(True)
else:
    print(result)

7593258303979883720
True


In [4]:
senha = "johnny*03"
hashed = bcrypt.hashpw(senha, bcrypt.gensalt())

TypeError: Unicode-objects must be encoded before hashing